PARTE 1

In [1]:
import os
# Forzamos a TensorFlow a usar el motor antiguo de Keras 2
os.environ["TF_USE_LEGACY_KERAS"] = "1"

import tensorflow as tf
import tf_keras as keras  # Usamos el paquete de compatibilidad
import transformers

print(f"Versión de TF: {tf.__version__}")
# Debería mostrar algo como 2.16.x o 2.17.x pero usando el motor de tf-keras

2026-01-25 02:41:50.421368: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Versión de TF: 2.16.2


In [2]:

import pandas as pd
import numpy as np
import re
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report
from sentence_transformers import SentenceTransformer

# ==========================================
# 0. Carga y Preprocesamiento de Datos
# ==========================================
df = pd.read_csv('retail_reviews.csv')

def clean_text(text):
    if pd.isna(text): return ""
    # Eliminar caracteres especiales tipo #@#&* y números
    text = re.sub(r'[^a-zA-ZáéíóúñÁÉÍÓÚÑ ]', '', text)
    # Convertir a minúsculas y quitar espacios extra
    return text.lower().strip()

df['text_clean'] = df['text'].apply(clean_text)
# Eliminar filas vacías tras la limpieza
df = df[df['text_clean'] != ""]

# ==========================================
# 1.1 Análisis de Embeddings (Word2Vec)
# ==========================================

# Tokenización para Word2Vec
tokenized_corpus = [doc.split() for doc in df['text_clean']]

# Entrenamiento del modelo Word2Vec
# vector_size: dimensión del vector, window: contexto, min_count: frecuencia mínima
w2v_model = Word2Vec(sentences=tokenized_corpus, vector_size=100, window=5, min_count=1, workers=4)

print("--- 1.1.a) Términos Similares ---")
palabras_clave = ["defectuoso", "rápido"]
for palabra in palabras_clave:
    if palabra in w2v_model.wv:
        similares = w2v_model.wv.most_similar(palabra, topn=5)
        print(f"\nSimilares a '{palabra}':")
        for p, sim in similares:
            print(f" - {p}: {sim:.4f}")

print("\n--- 1.1.c) Álgebra Vectorial (Analogía Retail) ---")
# Analogía: "excelente" - "positivo" + "negativo" debería tender a algo como "malo" o "pésimo"
try:
    resultado_algebra = w2v_model.wv.most_similar(positive=['excelente', 'negativo'], negative=['positivo'], topn=1)
    print(f"Operación: 'excelente' - 'positivo' + 'negativo'")
    print(f"Resultado semántico: {resultado_algebra[0][0]} (Similitud: {resultado_algebra[0][1]:.4f})")
except KeyError as e:
    print(f"Error en analogía: {e}")

# ==========================================
# 1.2 Clasificación de Texto
# ==========================================

# Preparación de etiquetas (Label Encoding manual para Positivo/Negativo)
df['label'] = df['sentiment'].map({'Positivo': 1, 'Negativo': 0})
df = df.dropna(subset=['label']) # Limpiar si hay etiquetas mal formadas

X_train, X_test, y_train, y_test = train_test_split(
    df['text_clean'], df['label'], test_size=0.2, random_state=42
)

# --- Enfoque A: Baseline TF-IDF + Regresión Logística ---
tfidf = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

model_lr = LogisticRegression()
model_lr.fit(X_train_tfidf, y_train)
y_pred_tfidf = model_lr.predict(X_test_tfidf)
f1_tfidf = f1_score(y_test, y_pred_tfidf)

# --- Enfoque B: BERT Embeddings + Regresión Logística ---
# Usamos un modelo ligero de BERT (paraphrase-multilingual-MiniLM-L12-v2)
print("\nGenerando embeddings de BERT (esto puede tardar un poco)...")
bert_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

X_train_bert = bert_model.encode(X_train.tolist())
X_test_bert = bert_model.encode(X_test.tolist())

model_bert = LogisticRegression()
model_bert.fit(X_train_bert, y_train)
y_pred_bert = model_bert.predict(X_test_bert)
f1_bert = f1_score(y_test, y_pred_bert)

# ==========================================
# Reporte Final
# ==========================================
print("\n" + "="*30)
print("REPORTE DE COMPARACIÓN")
print("="*30)
print(f"F1-Score TF-IDF (Baseline): {f1_tfidf:.4f}")
print(f"F1-Score BERT Embeddings:    {f1_bert:.4f}")
print("-" * 30)

# Identificar un caso donde TF-IDF falla y BERT acierta
print("\n--- Análisis de Fallos Específicos ---")
for i in range(len(y_test)):
    idx = y_test.index[i]
    if y_pred_tfidf[i] != y_test.iloc[i] and y_pred_bert[i] == y_test.iloc[i]:
        print(f"Texto: '{df.loc[idx, 'text']}'")
        print(f"Real: {y_test.iloc[i]} | TF-IDF predijo: {y_pred_tfidf[i]} | BERT predijo: {y_pred_bert[i]}")
        break

--- 1.1.a) Términos Similares ---

Similares a 'defectuoso':
 - insatisfecho: 0.9882
 - una: 0.9869
 - total: 0.9857
 - sucio: 0.9847
 - empaque: 0.9816

Similares a 'rápido':
 - recomendado: 0.9850
 - llegó: 0.9830
 - estado: 0.9817
 - eficaz: 0.9810
 - y: 0.9784

--- 1.1.c) Álgebra Vectorial (Analogía Retail) ---
Error en analogía: "Key 'negativo' not present in vocabulary"

Generando embeddings de BERT (esto puede tardar un poco)...

REPORTE DE COMPARACIÓN
F1-Score TF-IDF (Baseline): 0.9000
F1-Score BERT Embeddings:    0.8924
------------------------------

--- Análisis de Fallos Específicos ---
Texto: '  Producto duradero pero el precio es elevado.  '
Real: 0.0 | TF-IDF predijo: 1.0 | BERT predijo: 0.0
